In [ ]:
%%capture --no-stderr
%pip install --upgrade --quiet langchain  langchain-community langchainhub langchain-chroma beautifulsoup4

In [ ]:
import os

In [ ]:
pip install langchain-google-genai

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]=""
os.environ["LANGCHAIN_PROJECT"]="RAG WITH MEMORY"
os.environ["GOOGLE_API_KEY"]=""

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
gemini_embeddings=GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model="gemini-1.5-pro-001",convert_system_message_to_human=True)

In [ ]:
print(model.invoke("hi").content)

In [ ]:
import bs4
from langchain import hub

In [ ]:
from langchain.chains import create_retrieval_chain

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [ ]:
from langchain_chroma import Chroma

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
from langchain_core.prompts import MessagesPlaceholder

In [ ]:
loader=WebBaseLoader(web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/",),bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-content","post-title","post-header"))),)

In [ ]:
doc= loader.load()

In [ ]:
doc

In [ ]:
text_spliter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits=text_spliter.split_documents(doc)

In [ ]:
vectorstore = Chroma.from_documents(documents=splits, embedding=gemini_embeddings)
retriever = vectorstore.as_retriever()

In [ ]:
retriever

In [ ]:
system_prompt=(
    "You are an assistant for question-answering task."
    "Use the following pieces of retrieved context to answer the question"
    "If you don't know the answer, say that you don't know"
    "Use three sentence maximum and keep  the answer concise. "
    "\n\n"
    "{context}"

)


In [ ]:
chat_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)


In [ ]:
question_answering_chain=create_stuff_documents_chain(model,chat_prompt)

In [ ]:
rag_chain=create_retrieval_chain(retriever,question_answering_chain)

In [ ]:
respose=rag_chain.invoke({"input":"what is MRKL?"})

In [ ]:
respose["answer"]

In [ ]:
from langchain.chains import create_history_aware_retriever

In [ ]:
retriever_prompt = (
    "Given a chat history and the latest user question which might reference context in the chat history,"
    "formulate a standalone question which can be understood without the chat history."
    "Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
)

In [ ]:
contextualize_q_prompt=ChatPromptTemplate.from_messages(
    [
        ("system",retriever_prompt),
        ("human","{input}")
    ]


)

In [ ]:
history_aware_retriever=create_history_aware_retriever(model,retriever,contextualize_q_prompt)

In [ ]:
from langchain.chains import create_retrieval_chain

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [ ]:
qa_prompt=ChatPromptTemplate.from_messages(
    [
    ("system",system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human","{input}")
    ]
)

In [ ]:
question_answering_chain=create_stuff_documents_chain(model,qa_prompt)

In [ ]:
rag_chain=create_retrieval_chain(history_aware_retriever,question_answering_chain)

In [ ]:
from langchain_core.messages import HumanMessage,AIMessage

In [ ]:
chat_history=[]

In [ ]:
questions1="what is Task Decomposition ?"

In [ ]:
message1=rag_chain.invoke({"input":questions1,"chat_history":chat_history})

In [ ]:
message1["answer"]

In [ ]:
chat_history.extend(
    [
        HumanMessage(content=questions1),
        AIMessage(content=message1["answer"])
    ]
)

In [ ]:
chat_history

In [ ]:
second_question="what are common ways of doing it?"
message2=rag_chain.invoke({"input":second_question,"chat_history":chat_history})
print(message2["answer"])

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

In [ ]:
store={}

In [ ]:
def get_session_history(session_id:str)->BaseChatMessageHistory:
  if session_id not in store:
    store[session_id]=ChatMessageHistory()
  return store[session_id]

In [ ]:
conversational_rag_chain=RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"

)

In [ ]:
conversational_rag_chain.invoke(
    {"input":"What is Task Decomposition?"},
    config={"configurable":{"session_id":"abc123"}},
    )["answer"]

In [ ]:
store

In [ ]:
conversational_rag_chain.invoke(
    {"input":"what are common ways of doing it?"},
    config={"configurable":{"session_id":"abc123"}},
    )["answer"]

In [ ]:
for message in store["abc123"].messages:
  if isinstance(message,AIMessage):
    prefix="AI"
  else:
    prefix="Human"
  print(f"{prefix}:{message.content}\n")

In [ ]:
conversational_rag_chain.invoke(
    {"input":"What is a prompt technique like step xyz"},
    config={"configurable":{"session_id":"abc123"}},
    )["answer"]

In [ ]:
store

In [ ]:
for message in store["abc123"].messages:
  if isinstance(message,AIMessage):
    prefix="AI"
  else:
    prefix="Human"
  print(f"{prefix}:{message.content}\n")